In [ ]:
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

class Recommender:
    def __init__(user, searchdata='searches.txt'):
        user.searchdata = searchdata
        user.searches = self._load_searches()
        user.vectorizer = TfidfVectorizer(stop_words='english')
        user._update_vectors()

    def _load_searches(user):
        if not os.path.exists(user.searchdata):
            return []
        with open(user.history_file, 'r') as f:
            return [line.strip() for line in f.readlines() if line.strip()]

    def _save_search(user, query):
        with open(user.searchdata, 'a') as f:
            f.write(query + '\n')

    def _update_vectors(user):
        if user.searches:
            user.vectors = user.vectorizer.fit_transform(user.searches)
        else:
            user.vectors = None

    def add(user, query):
        user.searches.append(query)
        user._save_search(query)
        user._update_vectors()

    def recommend(user, query=None, top_n=5):
        if not user.vectors:
            return []

        vec = user.vectorizer.transform([query])
        scores = cosine_similarity(vec, user.vectors).flatten()
        indices = scores.argsort()[::-1]

        return [user.searches[i] for i in indices if user.searches[i] != query][:top_n]
